**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [37]:
import keras
import numpy as np
import random
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [38]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [62]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4, epsilon_min=0.01, epsilon_decay = 0.95):
        self.epsilon = epsilon
        self.n_action = n_action
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
    
    def set_epsilon(self, e):
        self.epsilon = e
        
        return self.epsilon

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

$\epsilon$ is essential for the exploration part.
Without it our agent would only take the first viable solutions it found without exploring/looking for a better one.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [40]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [41]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=11 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

board is the board game on which we play and it is used to determine the rewards that can be obtained at a certain position.

Position is also an array the size of the board and is used to determine where our agent can see

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [14]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [42]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [16]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 13.0/16.0. Average score (-3.0)
Win/lose count 9.5/11.0. Average score (-2.25)
Win/lose count 6.5/14.0. Average score (-4.0)
Win/lose count 10.5/22.0. Average score (-5.875)
Win/lose count 8.0/13.0. Average score (-5.7)
Win/lose count 12.0/21.0. Average score (-6.25)
Win/lose count 14.0/21.0. Average score (-6.357142857142857)
Win/lose count 8.0/17.0. Average score (-6.6875)
Win/lose count 8.0/18.0. Average score (-7.055555555555555)
Win/lose count 9.5/15.0. Average score (-6.9)
Win/lose count 8.5/18.0. Average score (-7.136363636363637)
Final score: -7.136363636363637


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Question 5 - Answers__

a) As stated above the Q value function is defined by:
\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>
\end{equation*}

We can deduce:
\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[r(s_{t},a_{t}) + \gamma \sum_{t\leq T-1}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \>
\end{equation*}
\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[r(s_{t},a_{t}) + \gamma Q^\pi(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \>
\end{equation*}

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

b)

By definition the optimal Q function is the Q function maximizing the reward, so:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

Using the previous equation, we obtain:
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [43]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory: 
            self.memory = self.memory[1:]

    def random_access(self):
        minibatch = random.sample(self.memory, 1)
        return minibatch

***
The pipeline we will use for training is given below:

In [18]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [53]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        
        self.dim = 5 * 5 * self.n_state
        
    def learned_act(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(4)
        
        predicted_Q = self.model.predict(s.reshape(1,5,5,self.n_state))
        self.avgQ = np.mean(predicted_Q)
        return np.argmax(predicted_Q)


    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        #assert (len(s_)== 5*5 * self.n_state and len(n_s)==5*5 *self.n_state)
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5, self.n_state))
        target_q = np.zeros((self.batch_size, self.n_action))

        #true_batch = min(self.batch_size, len(self.memory))
        
        for i in range(self.batch_size):
            state_, n_state_, action_, reward_, game_over_ = self.memory.random_access()[0]

            input_states[i] = state_
            #k = state_.reshape(1, self.dim)
            
            # get artificial targets for the actions
            target_q[i] = self.model.predict(state_.reshape(1,5,5, self.n_state))

            # updating real target for action of interest `a_`
            if game_over_:
                target_q[i][action_] = reward_
            else:
                #maxi = np.max(self.model.predict(n_state_.reshape(1, 5, 5, self.n_state)))
                target_q[i][action_] = (reward_ + self.discount * 
                                        np.max(self.model.predict(n_state_.reshape(1, 5, 5, self.n_state))))

 
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch((input_states), target_q)
        
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)

        input_shape = (5,5,self.n_state,)
        model = Sequential()
        model.add(Dense(512, activation='relu', input_shape=input_shape))
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(self.n_action, activation='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        
        self.model = model
        

In [20]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 000/011 | Loss 0.0034 | Win/lose count 2.0/5.0 (-3.0)
Epoch 001/011 | Loss 0.0049 | Win/lose count 1.5/4.0 (-2.5)
Epoch 002/011 | Loss 0.0067 | Win/lose count 9.5/4.0 (5.5)
Epoch 003/011 | Loss 0.0550 | Win/lose count 1.5/0 (1.5)
Epoch 004/011 | Loss 0.0040 | Win/lose count 0.5/1.0 (-0.5)
Epoch 005/011 | Loss 0.0019 | Win/lose count 3.5/5.0 (-1.5)
Epoch 006/011 | Loss 0.0060 | Win/lose count 5.0/8.0 (-3.0)
Epoch 007/011 | Loss 0.0068 | Win/lose count 6.0/8.0 (-2.0)
Epoch 008/011 | Loss 0.0055 | Win/lose count 1.5/3.0 (-1.5)
Epoch 009/011 | Loss 0.0012 | Win/lose count 2.5/4.0 (-1.5)
Epoch 010/011 | Loss 0.0036 | Win/lose count 8.0/1.0 (7.0)


In [21]:
E = Memory(100)
rand1 = np.zeros(5)
E.remember(rand1)
rand2 = np.ones(5)
E.remember([int(0)]*5)
E.remember([1]*5)
E.remember([2]*5)
E.remember([3]*5)
j_0, j_1, j_2, j_3, j_4 = E.random_access()[0]
print(type(j_0))
print(j_0)

<class 'int'>
3


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [54]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        input_shape = (5,5,self.n_state,)
        model = Sequential()
        model.add(Conv2D(64,kernel_size = (3,3), activation='relu', input_shape=input_shape))
        model.add(Conv2D(64, kernel_size = (3,3), activation='relu'))
        
        model.add(MaxPooling2D(pool_size=(1, 1)))
        model.add(Flatten())
        model.add(Dense(self.n_action, activation='softmax'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [23]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/011 | Loss 0.0019 | Win/lose count 2.5/3.0 (-0.5)
Epoch 001/011 | Loss 0.0701 | Win/lose count 1.5/2.0 (-0.5)
Epoch 002/011 | Loss 0.0026 | Win/lose count 4.5/6.0 (-1.5)
Epoch 003/011 | Loss 0.0048 | Win/lose count 2.5/3.0 (-0.5)
Epoch 004/011 | Loss 0.0044 | Win/lose count 1.5/2.0 (-0.5)
Epoch 005/011 | Loss 0.0016 | Win/lose count 2.5/8.0 (-5.5)
Epoch 006/011 | Loss 0.0037 | Win/lose count 5.0/4.0 (1.0)
Epoch 007/011 | Loss 0.0043 | Win/lose count 5.5/3.0 (2.5)
Epoch 008/011 | Loss 0.0588 | Win/lose count 3.5/1.0 (2.5)
Epoch 009/011 | Loss 0.0753 | Win/lose count 5.0/1.0 (4.0)
Epoch 010/011 | Loss 0.0021 | Win/lose count 3.5/2.0 (1.5)


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [102]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.0/2.0. Average score (0.0)
Win/lose count 1.5/0. Average score (0.75)
Win/lose count 0.5/0. Average score (0.6666666666666666)
Win/lose count 0.5/0. Average score (0.625)
Win/lose count 1.5/0. Average score (0.8)
Win/lose count 1.0/1.0. Average score (0.6666666666666666)
Win/lose count 1.5/4.0. Average score (0.21428571428571427)
Win/lose count 1.0/0. Average score (0.3125)
Win/lose count 1.0/2.0. Average score (0.16666666666666666)
Win/lose count 1.5/4.0. Average score (-0.1)
Win/lose count 1.5/2.0. Average score (-0.13636363636363635)
Final score: -0.13636363636363635
Test of the FC
Win/lose count 1.5/2.0. Average score (-0.5)
Win/lose count 0/3.0. Average score (-1.75)
Win/lose count 1.5/0. Average score (-0.6666666666666666)
Win/lose count 0.5/0. Average score (-0.375)
Win/lose count 1.0/3.0. Average score (-0.7)
Win/lose count 0.5/1.0. Average score (-0.6666666666666666)
Win/lose count 0.5/3.0. Average score (-0.9285714285714286)
Win/lose count 0.5

In [103]:
HTML(display_videos('cnn_test10.mp4'))

In [104]:
HTML(display_videos('fc_test10.mp4'))

The main issue is the lack of exploration, the agent always gets stuck in a corner. 
It is expecially true for the CNN agent and a bit less with the FC. But the FC agent performs overall way worse if the temperature is low.

To improve score we can:
 - Force a period of exploration 
 - Try other networks architecture

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [95]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        #Epsilon decay method
        if agent.epsilon >0.05: 
            agent.set_epsilon(agent.epsilon-(1/((e+1)*2)))
                              
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose


        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | epsilon ({})"
              .format(e, epoch, loss, win, lose, win-lose, round(agent.epsilon, 3)))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
        
class EnvironmentExploring(Environment):

    def __init__(self, grid_size=10, max_time=500, temperature=0.1,penalty = 0.25):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        self.penalty = penalty
        
        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
                              
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        reward = 0
        
        if train:
            reward = -self.malus_position[self.x, self.y]
        
        self.malus_position[self.x, self.y] = self.penalty
        
        self.t = self.t + 1
        reward = reward + self.board[self.x, self.y]
        
        #self.visited[self.x, self.y] += 1
        #self.visit_old_place_pen[self.x, self.y] = -self.penalty*self.visited[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
                              
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [96]:
T=500
size = 10
epochs_train =100
epochs = 11
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.7, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
#test(agent, env, epochs, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0106 | Win/lose count 13.5/120.0 (-106.5) | epsilon (0.665)
Epoch 001/100 | Loss 0.0157 | Win/lose count 15.0/118.0 (-103.0) | epsilon (0.632)
Epoch 002/100 | Loss 0.0203 | Win/lose count 11.5/116.0 (-104.5) | epsilon (0.6)
Epoch 003/100 | Loss 0.0251 | Win/lose count 14.0/124.0 (-110.0) | epsilon (0.57)
Epoch 004/100 | Loss 0.0121 | Win/lose count 12.5/125.5 (-113.0) | epsilon (0.542)
Epoch 005/100 | Loss 0.0214 | Win/lose count 17.0/117.0 (-100.0) | epsilon (0.515)
Epoch 006/100 | Loss 0.0199 | Win/lose count 13.0/117.5 (-104.5) | epsilon (0.489)
Epoch 007/100 | Loss 0.0167 | Win/lose count 10.5/124.25 (-113.75) | epsilon (0.464)
Epoch 008/100 | Loss 0.0111 | Win/lose count 8.0/125.0 (-117.0) | epsilon (0.441)
Epoch 009/100 | Loss 0.0112 | Win/lose count 16.0/123.0 (-107.0) | epsilon (0.419)
Epoch 010/100 | Loss 0.0129 | Win/lose count 10.5/122.5 (-112.0) | epsilon (0.398)
Epoch 011/100 | Loss 0.0254 | Win/lose count 8.0/125.75 (-117.75) | epsilon (0.378)
Epoch

KeyboardInterrupt: 

In [98]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 11.5/6.0. Average score (5.5)
Win/lose count 11.0/5.0. Average score (5.75)
Win/lose count 13.0/6.0. Average score (6.166666666666667)
Win/lose count 12.5/4.0. Average score (6.75)
Win/lose count 16.5/5.0. Average score (7.7)
Win/lose count 11.0/4.0. Average score (7.583333333333333)
Win/lose count 13.0/9.0. Average score (7.071428571428571)
Win/lose count 14.0/4.0. Average score (7.4375)
Win/lose count 13.0/5.0. Average score (7.5)
Win/lose count 14.5/7.0. Average score (7.5)
Win/lose count 11.5/12.0. Average score (6.7727272727272725)
Final score: 6.7727272727272725


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***